# Лабораторная работа №4
## Выполнил студент группы Семёнов Андрей Владиславович БФИ2001

# 

Реализовать следующие структуры данных:
  
● <b> Стек (stack): </b>  
Операции для стека: 
<i> инициализация, проверка на пустоту, добавление нового элемента в начало, извлечение элемента из начала;  </i>  
  
● <b> Дек (двусторонняя очередь, deque):  </b>  
Операции для дека: 
<i> инициализация, проверка на пустоту, добавление нового элемента в начало, добавление нового элемента в конец, извлечение элемента из начала, извлечение элемента из конца. </i>  
  
<i> Разработать программу обработки данных, содержащихся в заранее подготовленном txt-файле, в соответствии с заданиями, применив указанную в задании структуру данных. Результат работы программы вывести на экран и сохранить в отдельном txt-файле. </i>

In [1]:
from __future__ import annotations
from typing import Optional, Any
from dataclasses import dataclass

In [2]:
@dataclass
class Node:
    data: Any
    next_: Optional[Node] = None


class Stack:
    def __init__(self) -> None:
        self.head: Node = Node("head")
        self.size = 0

    def is_empty(self) -> bool:
        return self.size == 0

    def add(self, data: Any) -> None:
        node: Node = Node(data)
        node.next_ = self.head.next_
        self.head.next_ = node
        self.size += 1

    def pop(self) -> Stack:
        if self.is_empty():
            raise Exception("Stack is empty")
        res = self.head.next_
        self.head.next_ = self.head.next_.next_
        self.size -= 1
        return res.data

    def peek(self) -> Any:
        if self.is_empty():
            raise Exception("Stack is empty")
        return self.head.next_.data
    
    def __str__(self) -> str:
        root: Node = self.head.next_
        res: str = ""
        while root is not None:
            res += f"{root.data} "
            root = root.next_
        return res.rstrip()
        
    def __repr__(self) -> str:
        return self.__str__()

In [3]:
@dataclass
class DNode:
    data: Any
    next_: Optional[DNode] = None
    last_: Optional[DNode] = None


class Deque:
    def __init__(self) -> None:
        self.head: DNode = DNode("head")
        self.tail: DNode = DNode("tail")
        self.size: int = 0

    def is_empty(self) -> bool:
        return self.size == 0

    def add_first(self, data: Any) -> None:
        node: DNode = DNode(data)
        if self.is_empty():
            node.last_ = self.head
            node.next_ = self.tail
            self.head.next_ = node
            self.tail.last_ = node
        else:
            node.next_ = self.head.next_
            node.last_ = self.head
            self.head.next_ = node
        self.size += 1

    def add_last(self, data: Any) -> None:
        node: DNode = DNode(data)
        if self.is_empty():
            node.last_ = self.head
            node.next_ = self.tail
            self.head.next_ = node
            self.tail.last_ = node
        else:
            node.last_ = self.tail.last_
            self.tail.last_.next_ = node
            self.tail.last_ = node
        self.size += 1

    def pop_first(self) -> Any:
        if self.is_empty():
            raise Exception("Stack is empty")

        res: DNode = self.head.next_
        self.head.next_ = self.head.next_.next_
        self.size -= 1
        return res.data

    def pop_last(self) -> Any:
        if self.is_empty():
            raise Exception("Stack is empty")

        res: DNode = self.tail.last_
        self.tail.last_ = self.tail.last_.last_
        self.size -= 1
        return res.data

    def peek_last(self) -> Any:
        if self.is_empty():
            raise Exception("Stack is empty")
        return self.tail.last_.data

    def peek_first(self) -> Any:
        if self.is_empty():
            raise Exception("Stack is empty")
        return self.head.next_.data

    def __str__(self) -> str:
        root: DNode = self.head.next_
        res: str = ""
        while root is not None and root.data != "tail":
            res += f"{root.data} "
            root = root.next_
        return res.rstrip()
        
    def __repr__(self) -> str:
        return self.__str__()

In [15]:
def write_res_to_file(func):
    def inner(*args, **kwargs):
        res = func(*args, **kwargs)
        file_name = f"./lab4_txt_files/output/{func.__name__}.txt"
        with open(file_name, "w") as f:
            f.write(str(res))
        return res
    return inner

In [5]:
def read_txt(file_name: str) -> Deque:
    res: Deque = Deque()
    with open("./lab4_txt_files/input/" + file_name, "r") as f:
        for line in f.readlines():
            res.add_last(line.replace("\n", "").strip())
        print(f"{file_name.split('.')[0]} input: {res}")
    return res

### Задание №1
Отсортировать строки файла, содержащие названия книг, в алфавитном порядке с использованием двух деков.

In [6]:
@write_res_to_file
def solution1() -> str:
    inp: Deque = read_txt("task1.txt")
    tmp_deque: Deque = Deque()

    while not inp.is_empty():
        elem: str = inp.pop_last()

        while not tmp_deque.is_empty() and tmp_deque.peek_last() > elem:
            inp.add_last(tmp_deque.pop_last())

        tmp_deque.add_last(elem)

    return str(tmp_deque)


print(solution1())

task1 input: f e d c b a
a b c d e f


### Задание №2
Дек содержит последовательность символов для шифровки сообщений. Дан текстовый файл, содержащий зашифрованное сообщение. Пользуясь деком, расшифровать текст. 
Известно, что при шифровке каждый символ сообщения заменялся следующим за ним в деке по часовой стрелке через один.

In [7]:
@write_res_to_file
def solution2() -> str:
    text: str = str(read_txt("task2_text.txt"))
    res: list[str] = ["#" for _ in range(len(text))]
    dictionary: Deque = read_txt("task2_dict.txt")
    
    for _ in range(dictionary.size):
        l1 = dictionary.pop_first()
        l2 = dictionary.pop_first()

        for i, letter in enumerate(text):
            if letter == str(l1):
                res[i] = str(dictionary.peek_first())

        dictionary.add_first(l2)
        dictionary.add_last(l1)

    return ''.join(res)


print(solution2())

task2_text input: 11223344
task2_dict input: 1 2 3 4
33441122


### Задание №3
Даны три стержня и n дисков различного размера. Диски можно надевать на стержни, образуя из них башни. Перенести n дисков со стержня А на стержень С, сохранив их первоначальный порядок. При переносе дисков необходимо соблюдать следующие правила:
- на каждом шаге со стержня на стержень переносить только один диск;
- диск нельзя помещать на диск меньшего размера;
- для промежуточного хранения можно использовать стержень В.
Реализовать алгоритм, используя три стека вместо стержней А, В, С. Информация о дисках хранится в исходном файле.

In [8]:
@write_res_to_file
def solution3() -> Deque:
    n: int = int(read_txt("task3.txt").pop_first())
    a, b, c = Stack(), Stack(), Stack()
    for i in range(n, 0, -1):
        a.add(i)
    move(n, a, b, c)
    return c


def move(rings_count, a, b, c) -> None:
    if rings_count == 1:
        c.add(a.pop())
        return

    move(rings_count - 1, a, c, b)
    c.add(a.pop())
    move(rings_count - 1, b, a, c)
   

print(solution3())

task3 input: 9
1 2 3 4 5 6 7 8 9


### Задание №4
Дан текстовый файл с программой на алгоритмическом языке. За один просмотр файла проверить баланс круглых скобок в тексте, используя стек.

In [9]:
@write_res_to_file
def solution4() -> bool:
    txt: str = str(read_txt("task4.txt"))
    s: Stack = Stack()

    for i in txt:
        if i in ("(", ")"):
            if s.is_empty() or s.peek() == i:
                s.add(i)
            else:
                s.pop()

    return s.is_empty()


print(solution4())

task4 input: def main() -> None: print("Hello, world")) a: list[int] = [1, 2, 3]
False


### Задание №5
Дан текстовый файл с программой на алгоритмическом языке. За один просмотр файла проверить баланс квадратных скобок в тексте, используя дек.

In [10]:
@write_res_to_file
def solution5() -> bool:
    txt: str = str(read_txt("task4.txt"))
    d: Deque = Deque()

    for i in txt:
        if i in ("[", "]"):
            if d.is_empty() or d.peek_first() == i:
                d.add_first(i)
            else:
                d.pop_first()

    return d.is_empty()


print(solution5())

task4 input: def main() -> None: print("Hello, world")) a: list[int] = [1, 2, 3]
True


### Задание №6
Дан файл из символов. Используя стек, за один просмотр файла напечатать сначала все цифры, затем все буквы, и, наконец, все остальные символы, сохраняя исходный порядок в каждой группе символов.

In [11]:
@write_res_to_file
def solution6() -> str:
    letters: Stack = Stack()
    symbols: Stack = Stack()

    txt: str = str(read_txt("task6.txt"))
    res: str = ""
    for i in txt:
        if 48 <= ord(i) <= 57:
            res += i
        elif 97 <= ord(i) <= 122 or 65 <= ord(i) <= 90:
            letters.add(i)
        else:
            symbols.add(i)

    letters_res = ""
    while not letters.is_empty():
        letters_res += letters.pop()

    symbols_res = ""
    while not symbols.is_empty():
        symbols_res += symbols.pop()

    return res + letters_res[::-1] + symbols_res[::-1]


print(solution6())

task6 input: AsFa214@#$3SsdfASD*&(*^13
214313AsFaSsdfASD@#$*&(*^


### Задание №7
Дан файл из целых чисел. Используя дек, за один просмотр файла напечатать сначала все отрицательные числа, затем все положительные числа, сохраняя исходный порядок в каждой группе.

In [22]:
@write_res_to_file
def solution7() -> str:
    a: str = str(read_txt("task7.txt"))
    d: Deque = Deque()
    res = ""

    for i in a.split():
        if int(i) < 0:
            res += str(int(i)) + " "
        else:
            d.add_last(int(i))
        
    return res + str(d)


print(solution7())

task7 input: 0 -1 -4 3 2 5 -6 -8 1
-1 -4 -6 -8 0 3 2 5 1


### Задание №8
Дан текстовый файл. Используя стек, сформировать новый текстовый файл, содержащий строки исходного файла, записанные в обратном порядке: первая строка становится последней, вторая – предпоследней и т.д.

In [14]:
@write_res_to_file
def solution8() -> str:
    txt: str = str(read_txt("task8.txt")).split()
    s: Stack = Stack()
    for row in txt:
        s.add(row)

    res = ""
    for _ in range(len(txt)):
        res += s.pop() + " "
    return res


print(solution8())

task8 input: row1 row2 row3 row4 row5
row5 row4 row3 row2 row1 


### Вывод


В данной лабораторной работе были написаны две колекции - Стек и Дек. Кроме этого, были решены задачи на использование данных структур данных и отработаны навыки работы с файлами в языке python